In [2]:
import pandas as pd ; 
import numpy as np 
import time , os , sys , random
import sqlite3 , re , copy 
import matplotlib.pyplot as plt


def convert_db_data_to_csv(table_name , connection  , path = './data/' , filename="mydata.csv" ):
    df = pd.DataFrame(pd.read_sql('select * from '+table_name , connection)) ; 
    df.to_csv(os.path.join(path , filename)) ;
    

In [3]:
class allotment:
    
    def __init__(self , subjectid ,  facultyid1 , facultyid2 , roomid , dayno , hour , subjectname='' ):
        self.subjectid =subjectid ; 
        self.subjectname = subjectname
        self.facultyid1 = facultyid1
        self.facultyid2 = facultyid2
        self.roomid = roomid
        self.dayno = dayno
        self.hour = hour
    
    

class faculty:
    def __init__(self , fid , fname , ftype):
        self.id , self.name , self.type = fid , fname , ftype ; 
    def __str__(self):
        return "< ID : {} , Name : {}  , type : {}  >".format(self.id , self.name , self.type) ;
    def __repr__(self):
        return self.__str__() ;
        
        


class Timetable:
    
    def __init__(self):
        self.faculty_data = pd.read_csv("data/faculty.csv") ; 
        self.room_data = pd.read_csv('data/room.csv') ; 
        self.subject_data = pd.read_csv('data/subject.csv') ;
        self.test_data = pd.read_csv('data/test.csv') ;
        
        self.faculty_data = self.faculty_data[['facultyId' , 'facultyName' , 'facultyType']]
        self.room_data = self.room_data[['roomId' , 'roomType' , 'roomNo']]
        self.subject_data = self.subject_data[['subjectId' , 'subjectName' , 'subjectType' , 'credits' , 'duration']]
        self.test_data = self.test_data[['batchID' , 'subjectID' , 'facultyID1' , 'facultyID2']]
        
        
#         type : pd.DataFrame
        self.normal_rooms = self.room_data[(self.room_data.roomType==0) & (self.room_data.roomNo)]
        self.lab_rooms = self.room_data[(self.room_data.roomType==1) & (self.room_data.roomNo)]

        
#         lab_subjects : type = np.array
        self.lab_subjects = self.subject_data[self.subject_data.subjectType==1].subjectId.values
        self.normal_subjects = self.subject_data[self.subject_data.subjectType==0].dropna().subjectId.values
        
#         type:np.array
        self.sectionslist = self.test_data[self.test_data.subjectID.apply(lambda sid : True if sid in self.normal_subjects else False)].batchID.unique()
    
    
#       type: dict , map : string -> np.array
        self.section_to_subjects = {}
    
#     type: dict , map : string -> np.array
        self.batch_to_labs = {} 
        
                
        
#         method calls
        self.populate_section_to_subject() ; 
        self.create_faculty_objects() ; 
        self.init_faculties() ;
        
        
        
#         returns facultyid if he is free else None
    def get_free_faculty(self , facultylist , day , hour ):
        '''Returns a random faculty from a np.array list of faculties who is free at that day and hour '''
        
        faculties = facultylist.values
        
        try:
            for faculty in faculties:
                if(self.faculty_to_day_hour_slot_map.get(faculty).get(day).get(hour).get('alloted') == False):
                    self.faculty_to_day_hour_slot_map.get(faculty).get(day).get(hour)['alloted'] = True
                    return faculty ;
        
        except AttributeError as err:
            print(err , 'faculty = ' , faculty , ' ,  day , hour = ' , day , hour ) ;  
        except Exception as Exp:
            print(Exp) ; 
            
    
    def display_time_table(self , section):
        asec = obj.non_lab_allotment[obj.non_lab_allotment.section=='4A']
        asec = asec.drop(['section' , 'roomno'], axis=1) 
        asec = asec.set_index(['day' , 'hour'])
        asec['subjectname'] = asec.subjectid.apply(lambda x : obj.subject_data[obj.subject_data.subjectId==x].values[0][1])
        asec[['subjectname' , 'facultyid']].apply(lambda x : x.subjectname + ' >> ' +  str(x.facultyid)  , axis=1).unstack()

            
        
        
    
#     populates the section_to_subjects dict  , maps section string to subjects list
    def populate_section_to_subject(self):
        for section in self.sectionslist:
            self.section_to_subjects[section] = self.test_data[self.test_data.batchID==section].subjectID ;
            
    
        
    def init_faculties(self):
        '''Initialize the faculties to be not allocated to all the hours of all days'''        
        self.faculty_to_day_hour_slot_map = {}
        day_to_hour = {}
        faculty_to_data = {} 

        hour_to_allotflag = {}
        for hour in range(1 , 9):
            hour_to_allotflag.update({hour : copy.deepcopy({'alloted': False})}) ;

        for day in ['mon' , 'tue' , 'wed' , 'thu' , 'fri' , 'sat']:
            day_to_hour.update({day : copy.deepcopy(hour_to_allotflag)}) ;

        for faculty_object in self.faculties:
            self.faculty_to_day_hour_slot_map.update({faculty_object.id : copy.deepcopy(day_to_hour)}) ;
        
                    
#     return type : None
    def allot_slots_normal_class(self):
        
        allotment = pd.DataFrame(columns=['day' ,'section', 'hour', 'subjectid' , 'roomno' , 'facultyid']) ;
        
        for section in self.sectionslist:
            selected_room = self.normal_rooms.roomNo.sample().values[0] ;
            section_subjects_original = self.section_to_subjects[section].values ;
            
            for day in ['mon' , 'tue' , 'wed' , 'thu' , 'fri' , 'sat']:
                batch_subs = section_subjects_original ; 

                for hour in range(1 , 9):
                    if(not batch_subs.any()):
                        selected_subject = self.section_to_subjects[section].sample().values[0]
                    else:
                        selected_subject_index =  random.randrange(len(batch_subs));
                        selected_subject = batch_subs[selected_subject_index] ; 
                        batch_subs = np.delete(batch_subs , selected_subject_index) ;
                        
                    
                    selected_faculty = self.get_free_faculty(self.test_data[self.test_data['subjectID']==selected_subject].facultyID1 , day= day , hour = hour) 
                    
                    allotment = allotment.append({'section' : section , 'day' : day , 'hour' : hour  , 'subjectid' : selected_subject , 'roomno' : selected_room , 'facultyid' : selected_faculty  } , ignore_index=True)
                    
        self.non_lab_allotment = allotment ;
                    
    
    
    def create_faculty_objects(self):
        '''Create facultie objects using the data ''' 
        self.faculties = set() ;
        for i in range(self.faculty_data.count().facultyId):
            self.faculties.add(faculty(self.faculty_data.loc[i].facultyId , self.faculty_data.loc[i].facultyName , self.faculty_data.loc[i].facultyType))
                
        
                    
    
    def check_allotment_validity(self , allotment_dataframe = None ):
        '''checks if the allotment DataFrame satisfies the hard constraints '''
        if(not allotment_dataframe):
            allotment_dataframe = self.non_lab_allotment ; 
        
        df = allotment_dataframe ;
        print("Starting faculty duplicate check for each hour of some day ") ; 
        
        for day in ['mon' , 'tue' , 'wed' , 'thu' , 'fri' , 'sat']:
            for hour in range(1 , 9):
                temp = df[(df.hour==1) & (df.day==day)]
                temp = temp[temp.facultyid.apply(lambda x : True if x else False )].facultyid
                if(not (temp.nunique()==temp.count())):
                    print("Fails faculty non duplicate test : day , hour = " , day , hour) ; 
                    print("temp = " , temp) ;
                    print("returning .. ") ; 
                    return  False ;
        
        print("\nSUCCESS ^_^ ") ; 
            

obj = Timetable()

In [4]:
obj.allot_slots_normal_class()

In [5]:
obj.check_allotment_validity()

Starting faculty duplicate check for each hour of some day 

SUCCESS ^_^ 


In [8]:
obj.test_data[obj.test_data.subjectID.apply(lambda sid : True if sid in normal_subs else False)].batchID.unique()

array(['4A', '4B', '4C', '6A', '6B', '6C', '8', 'M-CNE', 'M-CSE'],
      dtype=object)

In [ ]:
obj.subject_data.drop('duration' , axis=1 , inplace=True)

In [89]:
asec = obj.non_lab_allotment[obj.non_lab_allotment.section=='4A']
asec = asec.drop(['section' , 'roomno'], axis=1) 
asec = asec.set_index(['day' , 'hour'])
asec['subjectname'] = asec.subjectid.apply(lambda x : obj.subject_data[obj.subject_data.subjectId==x].values[0][1])
asec[['subjectname' , 'facultyid']].apply(lambda x : x.subjectname + ' >> ' +  str(x.facultyid)  , axis=1).unstack()


hour,1,2,3,4,5,6,7,8
day,,,,,,,,
fri,Statistics and Probability Theory >> 46,Finite Automata & Formal Languages >> 21,Microcontroller >> 39,Analysis & Design of Algorithms >> 5,Graph Theory and Combinatorics >> 47,Object Oriented Programming with C++ >> 29,FREE >> 42,Statistics and Probability Theory >> 46
mon,Microcontroller >> 39,Object Oriented Programming with C++ >> 29,Statistics and Probability Theory >> 46,Finite Automata & Formal Languages >> 21,Graph Theory and Combinatorics >> 47,Analysis & Design of Algorithms >> 5,Const. of India & Prof. Ethics >> 48,FREE >> 42
sat,Statistics and Probability Theory >> 46,Statistics and Probability Theory >> 46,FREE >> 42,Const. of India & Prof. Ethics >> 48,Microcontroller >> 39,Graph Theory and Combinatorics >> 47,Analysis & Design of Algorithms >> 5,Finite Automata & Formal Languages >> 21
thu,Analysis & Design of Algorithms >> 5,FREE >> 42,Microcontroller >> 39,Statistics and Probability Theory >> 46,Graph Theory and Combinatorics >> 47,Finite Automata & Formal Languages >> 21,Const. of India & Prof. Ethics >> 48,Statistics and Probability Theory >> 46
tue,FREE >> 42,Statistics and Probability Theory >> 46,Graph Theory and Combinatorics >> 47,Analysis & Design of Algorithms >> 5,Finite Automata & Formal Languages >> 21,Const. of India & Prof. Ethics >> 48,Object Oriented Programming with C++ >> 29,Statistics and Probability Theory >> 46
wed,Object Oriented Programming with C++ >> 29,Const. of India & Prof. Ethics >> 48,Statistics and Probability Theory >> 46,Graph Theory and Combinatorics >> 47,Statistics and Probability Theory >> 46,Microcontroller >> 39,FREE >> 42,Finite Automata & Formal Languages >> 21


In [12]:
obj.test_data[obj.test_data.subjectID.apply(lambda x : x in obj.lab_subjects)].batchID.unique()

array(['4A1', '4A2', '4A3', '4B1', '4B2', '4B3', '4C1', '4C2', '4C3',
       '6A1', '6A2', '6A3', '6B1', '6B2', '6B3', '6C1', '6C2', '6C3'],
      dtype=object)